In [49]:
import init

import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import normalize

from dataset.graph_loader import GraphLoader

In [50]:
loader = GraphLoader(
    data_path='/Users/marvzhang/projects/crawlab-team/auto-html/data/quotes.toscrape.com/json/http___quotes_toscrape_com_.json', 
    embed_walk_length=3,
)

loader.run()

Graph(num_nodes=145, num_edges=145,
      ndata_schemes={'feat': Scheme(shape=(106,), dtype=torch.float32)}
      edata_schemes={})


In [51]:
loader.nodes_embedded_tensor

tensor([[112,   0, 134, 123],
        [123, 112,   0, 134],
        [134, 123, 112,   0],
        [  0, 134, 123, 112],
        [ 11,   0, 134, 123],
        [ 22,  11,   0, 134],
        [ 33, 134, 123, 112],
        [ 44,  33, 134, 123],
        [ 55,  44,  33, 134],
        [ 58, 123, 112,   0],
        [ 59,  58, 123, 112],
        [ 60,  59,  58, 123],
        [ 61,  60,  59,  58],
        [ 62,  60,  59,  58],
        [ 63,  62,  60,  59],
        [ 64,  62,  60,  59],
        [ 65,  60,  59,  58],
        [ 66,  65,  60,  59],
        [ 67,  65,  60,  59],
        [ 68,  65,  60,  59],
        [ 69,  65,  60,  59],
        [ 70,  65,  60,  59],
        [ 71,  59,  58, 123],
        [ 72,  71,  59,  58],
        [ 73,  71,  59,  58],
        [ 74,  73,  71,  59],
        [ 75,  73,  71,  59],
        [ 76,  71,  59,  58],
        [ 77,  76,  71,  59],
        [ 78,  76,  71,  59],
        [ 79,  76,  71,  59],
        [ 80,  59,  58, 123],
        [ 81,  80,  59,  58],
        [ 

In [52]:
idx = loader.nodes_embedded_tensor[10]
idx

tensor([ 59,  58, 123, 112])

In [53]:
loader.nodes_features_tensor[idx]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0

In [54]:
loader.nodes_features_tensor[idx].shape

torch.Size([4, 106])

In [55]:
loader.nodes_features_tensor[loader.nodes_embedded_tensor].shape

torch.Size([145, 4, 106])

In [56]:
loader.nodes_features_tensor[loader.nodes_embedded_tensor]

tensor([[[0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.]],

        [[0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]],

        [[0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 1., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0

## Nodes Features (tags + attributes)

In [57]:
feat_tags_attrs = loader.nodes_features_tensor
feat_tags_attrs

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], requires_grad=True)

In [58]:
X_feat_tags_attrs = feat_tags_attrs.detach().numpy()
X_feat_tags_attrs

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [59]:
X_feat_tags_attrs = normalize(X_feat_tags_attrs, norm='l1', axis=1)
X_feat_tags_attrs

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0.5, 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]], dtype=float32)

## Nodes Features (node2vec)

In [60]:
feat_node2vec = loader.nodes_features_tensor[loader.nodes_embedded_tensor].sum(dim=1)
feat_node2vec.shape

torch.Size([145, 106])

In [61]:
X_feat_node2vec = feat_node2vec.detach().numpy()
X_feat_node2vec.shape

(145, 106)

In [62]:
X_feat_node2vec = normalize(X_feat_node2vec, norm='l1', axis=1)
X_feat_node2vec

array([[0.        , 0.        , 0.        , ..., 0.        , 0.11111111,
        0.11111111],
       [0.        , 0.        , 0.        , ..., 0.        , 0.11111111,
        0.11111111],
       [0.        , 0.        , 0.        , ..., 0.        , 0.11111111,
        0.11111111],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.08333334],
       [0.        , 0.        , 0.        , ..., 0.        , 0.09090909,
        0.        ],
       [0.        , 0.07692308, 0.        , ..., 0.07692308, 0.        ,
        0.        ]], dtype=float32)

## Nodes Features (all: tags + attributes + node2vec)

In [63]:
X = np.concatenate((X_feat_tags_attrs, X_feat_node2vec), axis=1)
X.shape

(145, 212)

In [64]:
X = normalize(X, norm='l2', axis=1)
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.10540926,
        0.10540926],
       [0.        , 0.        , 0.        , ..., 0.        , 0.14213382,
        0.14213382],
       [0.        , 0.        , 0.        , ..., 0.        , 0.16666666,
        0.16666666],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.10660036],
       [0.        , 0.        , 0.        , ..., 0.        , 0.11508708,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.09925836, 0.        ,
        0.        ]], dtype=float32)

## DBSCAN Clustering

In [65]:
dbscan = DBSCAN()
dbscan.fit(X)

DBSCAN()

In [66]:
labels = pd.Series(dbscan.labels_)
labels_counts = labels.value_counts()
labels_counts[labels_counts.index > -1]

0    10
1    10
2    10
3    10
5    10
4     8
dtype: int64

In [67]:
for i in labels_counts[labels_counts.index > -1].index.tolist():
    top_label = labels_counts.index.values[i]
    top_label

    top_nodes_idx = labels[labels == top_label].index.values
    top_nodes_idx

    nodes = np.array(loader.nodes)[top_nodes_idx]
    nodes = sorted(nodes, key=lambda n: n.graph_node_id)
#     nodes = list(map(lambda n: f'[{loader.graphs_id_file_map.get(n.graph_id)}] {n.graph_node_id} -> {n.graph_parent_id}: {n.features}', nodes))
    nodes = list(map(lambda n: f'{n.graph_node_id} -> {n.graph_parent_id}: {n.features}', nodes))
    print(np.array(nodes[:50]))
    print('\n\n')

["0.10 -> 0.9: [('tag', 'div'), ('class', 'col-md-8')]"
 "0.103 -> 0.102: [('tag', 'small'), ('class', 'author'), ('itemprop', 'author')]"
 "0.104 -> 0.102: [('tag', 'a'), ('href', '/author/Eleanor-Roosevelt')]"
 "0.106 -> 0.105: [('tag', 'meta'), ('class', 'keywords'), ('itemprop', 'keywords'), ('content', 'misattributed-eleanor-roosevelt')]"
 "0.107 -> 0.105: [('tag', 'a'), ('class', 'tag'), ('href', '/tag/misattributed-eleanor-roosevelt/page/1/')]"
 "0.11 -> 0.10: [('tag', 'h1')]"
 "0.112 -> 0.110: [('tag', 'a'), ('href', '/author/Steve-Martin')]"
 "0.114 -> 0.113: [('tag', 'meta'), ('class', 'keywords'), ('itemprop', 'keywords'), ('content', 'humor,obvious,simile')]"
 "0.115 -> 0.113: [('tag', 'a'), ('class', 'tag'), ('href', '/tag/humor/page/1/')]"
 "0.116 -> 0.113: [('tag', 'a'), ('class', 'tag'), ('href', '/tag/obvious/page/1/')]"
 "0.117 -> 0.113: [('tag', 'a'), ('class', 'tag'), ('href', '/tag/simile/page/1/')]"
 "0.118 -> 0.17: [('tag', 'nav')]"
 "0.119 -> 0.118: [('tag', 'ul